In [ ]:
import numpy as np
import control as con
from numpy import linalg as LA

import cvxpy
import optim_tools #own file with helper

import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
print cvxpy.installed_solvers()

In [ ]:
###########################
# Hydraulischer Aktor     #
###########################

A0 = np.matrix([[0,   1,       0],
                [-10, -1.167, 25],
                [0,   0,    -0.8]])
print "Eigenvalues: {}".format(LA.eigvals(A0))
#a = -A[-1,:].T ### !!!!!
#print a
b0 = np.matrix([[0],[0],[2.4]])
c0 = np.matrix([1, 0, 0])
d0 = np.matrix([0])
u_max = 10.5
n = 3

X00 = [np.matrix([-20.0, -10.0, -10.0]).T,
       np.matrix([-20.0, -10.0, 10.0]).T,
       np.matrix([-20.0,  10.0, -10.0]).T,
       np.matrix([20.0,  -10.0, 10.0]).T,
       np.matrix([20.0,  -10.0, -10.0]).T,
       np.matrix([20.0,   10.0, 10.0]).T]

#print "A:\n", A
#print "a:\n", a
#print "b:\n", b
#print "c:\n", c

# Convert to Normalform
(A1, b1, c1, d1), T1, Q1 = optim_tools.get_Steuerungsnormalform(A0, b0, c0.T, d0)
a1 = -A1[-1][:].T #!!!!
print "T1:\n", T1

# Convert to Normalform
ss, T2 = con.canonical_form(con.ss(A0, b0, c0, d0), form='reachable')

assert np.allclose(T1*X00[1],
                   optim_tools.reverse_x_order(T2*X00[1])),\
"own Steuerungsnormalform Transformation not equal python control version"
#print "x_r1:\n", T1*X00[1]
#print "x_r2(backwards):\n", optim_tools.reverse_x_order(T2*X00[1])

A = optim_tools.reverse_x_order(np.matrix(ss.A))
a = -A[-1][:].T #!!!!

b = optim_tools.reverse_x_order(np.matrix(ss.B))
c = optim_tools.reverse_x_order(np.matrix(ss.C))
d = optim_tools.reverse_x_order(np.matrix(ss.D)) # == 0!

print "A:\n", A
assert np.allclose(A, A1)
print "a:\n", a
assert np.allclose(a, a1)
print "b:\n", b
assert np.allclose(b, b1)
print "c:\n", c
assert np.allclose(c, c1.T)

X0 = [T1.dot(x0) for x0 in X00]
print "X0:\n", X0
#print "A1:\n", A1
#print "a1:\n", a1
#print "b1:\n", b1
#print "c1:\n", c1

In [ ]:
def someparameters():
    k0 = np.matrix([[ 0.00702601],
     [-0.79417547],
     [-0.01583519]])

    k1 = np.matrix([[ 0.00044966],
     [-0.02280205],
     [-0.00489574]])

    k0_star = np.matrix([[ 0.00151984],
     [-0.2060622 ],
     [-0.00826113]])

    k1_star = np.matrix([[ 0.00045032],
     [-0.02262498],
     [-0.00470835]])


In [ ]:
# Solution directly from polyplacement char. polynomial to wanted polynomial (fastest) with given polynomical coeff a
def k_explizit(roots_p1, roots_pmin, pmin, a):
    n = len(roots_p1)
    
    k0 = np.zeros((n,))
    k1 = np.zeros((n,))

    a_tilde_p1 = np.matrix(np.poly(roots_p1)[:0:-1]).T # get the characteristical polynomial backwards
    a_tilde_pmin = np.matrix(np.poly(roots_pmin)[:0:-1]).T # get the characteristical polynomial backwards
    
    k1 = (a_tilde_p1 - a_tilde_pmin) / (1.0-1.0/pmin)
    k0 = a_tilde_p1 - a - k1

    return np.matrix(k0), np.matrix(k1)

%timeit k_explizit([1,2,4], [2,4,8], 0.1, a)

In [ ]:
# Do canonical form first (costy)
def k_explizit_x(roots_p1, roots_pmin, pmin, A, b):
    (A_R, _, _, _), _, _ = optim_tools.get_Steuerungsnormalform(A, b, b, 0)
    a = -A_R[-1][:].T
    n = len(roots_p1)
    
    k0 = np.zeros((n,))
    k1 = np.zeros((n,))

    a_tilde_p1 = np.matrix(np.poly(roots_p1)[:0:-1]).T # get the characteristical polynomial backwards
    a_tilde_pmin = np.matrix(np.poly(roots_pmin)[:0:-1]).T # get the characteristical polynomial backwards
    
    k1 = (a_tilde_p1 - a_tilde_pmin) / (1.0-1.0/pmin)
    k0 = a_tilde_p1 - a - k1

    return np.matrix(k0), np.matrix(k1)

%timeit k_explizit_x([1,2,4], [2,4,8], 0.1, A, b)

In [ ]:
# Use python control to place poles and then interpolate (faster with A and b)
def k_explizit2(roots_p1, roots_pmin, pmin, A, b):
    r0 = con.place(A, b, roots_p1) #k(p=1)
    r1 = con.place(A, b, roots_pmin) #k(p=pmin)

    # This seems to work as expected
    k1 = 1.0/(1.0-1.0/pmin) * (r0 - r1)
    k0 = r0 - k1
    return np.matrix(k0).T, np.matrix(k1).T

%timeit k_explizit2([1,2,4], [2,4,8], 0.1, A, b)

In [ ]:
### Design Sättigungsregler mittels konvexer Hülle (A.3)

# Variables (Convex)
#  Name in A.3 | Name in Program
#    Q  = P^⁻1 |   = Q
#    z  = Ql   |   = z0
#    z* = Ql*  |   = z1

# Variables (Quasi convex)
#    gamma     |   = g

# Parameter
#    mu        |   = m (Designparameter)
#    X0        |   = X0 = [x0,...]
#    A         |   = A
#    b         |   = b

# Initialize
n = len(b) # get dim of system

# Define Variables
#Q  = cvxpy.Semidef(n) #symmetric and positive semidefinite
Q  = cvxpy.Variable(n, n) # Semidef could go as an additional constraint as well, thereby no need fo Q to be symmetric

z0 = cvxpy.Variable(n)
z1 = cvxpy.Variable(n)

# Bisection parameter
g = cvxpy.Parameter(sign='positive')
#g.value = g_val # TODO set by bisection loop (function?)

m = cvxpy.Parameter(sign='positive')
#m.value = 1 # TODO: mu* >=1

# Define Constraints
const_sdQ = Q >> 0 # semidef but not (necessarily) symmetric

# (A.10)
const_A10 = [cvxpy.bmat([[Q,       X0[i]],
                         [X0[i].T, 1    ]]) >> 0
                            for i in range(0, len(X0))]

# (A.11)
const_A11 = cvxpy.bmat([[Q,  z0],
                        [z0.T, 1 ]]) >> 0

# (A.12)
const_A12 = cvxpy.bmat([[Q,    z1  ],
                        [z1.T,   m**2]]) >> 0

# (A.13)
const_A13 = Q*A + A.T*Q - b*z0.T - z0*b.T << 0 # This constraint is strict definit

# (A.14)
const_A14 = Q*A + A.T*Q - b*z1.T - z1*b.T << -2*g*Q # This constraint is strict definit
#const_A14 = Q*A + A.T*Q - b*z1.T - z1*b.T << cvxpy.bmat([[-2*g, 0, 0],
#                                                         [0, -2*g, 0],
#                                                         [0, 0, -2*g]])*Q # This constraint is strict definit


# Collect all constraints
constraints = [const_sdQ]
constraints.extend(const_A10) ##!! Beware of the "extend" if input is array
constraints.append(const_A12)
constraints.append(const_A13)
constraints.append(const_A14)


# Feasibility for bisection:
obj = cvxpy.Minimize(0)

# Stronger requirements for bisection? -> probably resulting in higher iterations, thus better results: TODO: Fix bisect!
obj_alt = cvxpy.Maximize(cvxpy.log_det(Q)) # Identical to geo_mean (in term of convexity and result)

# Form and solve problem.
prob = cvxpy.Problem(obj, constraints)

In [ ]:
"""
 Aemo of plotting complex functions in python.

 Jim M | Feb 2011 | GPL
"""
# Plotting functions ; see the example below
# and http://matplotlib.sourceforge.net/
#from matplotlib.pyplot import plot, legend

# Complex math (cmath) python functions ;
# see  see http://docs.python.org/library/cmath.html
#from cmath import sin, cos, exp, pi, log, polar, rect, phase, sqrt

# Note that python represents imaginary numbers like "3i" as "3j",
# where "j" meaning "sqrt(-1) must be preceded by a number,
# so "sqrt(-1)" alone would in python be "1j".
#
#     (3,4) complex rectangular form:     z = 3 + 4j
#     (x,y) complex rectangular form :    z = x + y * 1j
#     polar form :                        z = r * exp(1j * theta)
#     abs(z)   is length of complex number = r
#     phase(z) is angle of complex number = theta
#     z.real   is real part
#     z.imag   is imaginary part
#
# abs() is a python built-in; as are complex numbers themselves.
# But the other functions needed to be imported in their complex versions.
# The numeric constant pi can be imported from math or cmath.

# Remember that
# 1. lambda(x: ...) is an anyonymous function of x, e.g. lambda(x: 2*x+1)
# 2. map(f, [a, b, c, ...])  # returns [f(a), f(b), f(c), ...]


# == So here are a few utility functions for multiplying scalars and vectors.

# a scalar times a vector returns a vector
def scale_vector(scale, vector):
  result = [0]*len(vector)
  for i in range(len(result)):
    result[i] = scale * vector[i]
  return result

# dot product of two vectors = sum(x[0]*y[0] + ... + x[n-1]*y[n-1])
def vector_dot(vector1, vector2):
  result = 0
  for i in range(len(vector1)):
    result += vector1[i] * vector2[i]
  return result

# return real part of a vector
def real_vector(vector):
  return map(lambda x: x.real, vector)

# return imaginary part of a vector
def imag_vector(vector):
  return map(lambda x: x.imag, vector)

In [ ]:
# TEST k_explizit functions

# uboot
A_x = np.matrix([[0., 1., 0.],
              [0., 0., 1.],
              [0., 0., -0.005]])
a_x = -A_x[-1][:].T #!!!!

b_x = np.matrix([[0], [0], [1.]])

d_x = 0
c_x = np.matrix([[1], [0], [0]])

sys_x = con.ss(A_x, b_x, c_x.T, d_x)
#mag, phase, omega = bode(sys1)
#arr1, arr2 = control.step(sys)
#plt.plot(arr2, arr1)

#poles, zeros = control.matlab.pzmap(sys, True)
plt.axis([-5,.1,-3,3])
#plt.show

roots_p1_x = [-1, -1+1j, -1-1j]
roots_pmin_x = [-3, -3+2j, -3-2j]
pmin_x = 0.1

k0_x, k1_x = k_explizit(roots_p1_x, roots_pmin_x, pmin_x, a_x)

k0_x2, k1_x2 = k_explizit_x(roots_p1_x, roots_pmin_x, pmin_x, A_x, b_x)
k0_x3, k1_x3 = k_explizit2(roots_p1_x, roots_pmin_x, pmin_x, A_x, b_x)

#print k0_x, k1_x
#print k0_x2, k1_x2
#print k0_x3, k1_x3

assert np.allclose(k0_x, k0_x2)
assert np.allclose(k1_x, k1_x2)
assert np.allclose(k0_x2, k0_x3)
assert np.allclose(k1_x2, k1_x3)


poles = []
for p in np.arange(1, 0.09, -0.01):
    #sys_cl = control.ss(A-b*(r0), b, c.T, d)
    #poles2, zeros2 = control.matlab.pzmap(sys_cl, True)
    #sys_cl = control.ss(A-b*(r1), b, c.T, d)
    #poles2, zeros2 = control.matlab.pzmap(sys_cl, True)
    #print p
    sys_cl = con.ss(A-b*(k0_x3+1.0/p*k1_x3).T, b, c, d)
    pole, zeros = con.matlab.pzmap(sys_cl, True)
    #print pole[0]
    poles.append(pole)
    #print poles2
plt.show



In [ ]:
# TODO: zeta -> zeta0 AND zeta_* -> zeta1  ---> Should come from optimisation loop
zeta0 = 2.5  #More or less chosen randomly here
zeta1 = 5.0  #More or less chosen randomly here

m1 = 1.5 # mu_*
pmin = 0.01

In [ ]:
print "Entering step 1"

In [ ]:
%%time
# Perform bisection on g for mu=1
m.value = 1
[[o_Q, o_z0, o_z1], o_g] = optim_tools.bisect_max(0, None, prob, g, [Q, z0, z1], bisect_verbose=True,
                                              solver=cvxpy.SCS, warm_start=True, max_iters=800000, verbose=False)

print "-----------RESULTS-----------"
# l*(mu=1 -> m0) -> l1m0
l1m0 = LA.inv(o_Q) * o_z1
print "l*(mu=1)=\n", l1m0

roots_m0_p1 = LA.eigvals(A-b*l1m0.T)
print "lambda^{hat}(p=1)    = ", roots_m0_p1

roots_m0_pmin = zeta0 * roots_m0_p1.real + 1j*roots_m0_p1.imag
#roots_m0_pmin = zeta0 * roots_m0_p1  # Shifting on both real and imaginary axis in zeta

print "lambda^{hat}(p=pmin) = ", roots_m0_pmin

In [ ]:
%%time
# Perform bisection on g for mu>=1
m.value = m1 # TODO: mu* comes from optimisation loop

[[o_Q, o_z0, o_z1], o_g] = optim_tools.bisect_max(0, None, prob, g, [Q, z0, z1], bisect_verbose=True,
                                              solver=cvxpy.SCS, warm_start=True, max_iters=1600000, verbose=False)

print "-----------RESULTS-----------"
# l*(mu=1 -> m0) -> l1m0
l1m1 = LA.inv(o_Q) * o_z1
print "l*(mu=mu_*={})=\n".format(m.value), l1m1

roots_m1_p1 = LA.eigvals(A-b*l1m1.T)
print "lambda^{hat}_{*}(p=1)    = ", roots_m1_p1

roots_m1_pmin = zeta1 * roots_m1_p1.real + 1j*roots_m1_p1.imag
#roots_m0_pmin = zeta1 * roots_m1_p1  # Shifting on both real and imaginary axis in zeta

print "lambda^{hat}_{*}(p=pmin) = ", roots_m1_pmin

In [ ]:
# Get Polynomcoefficiants
# k0_i -> k_i

k0_0, k0_1 = k_explizit2(roots_m0_p1, roots_m0_pmin, pmin, A, b)

plt.axis([-2, .1, -3.5, 3.5])
poles = []
for p in np.arange(1, pmin, -0.01):
    #sys_cl = control.ss(A-b*(r0), b, c.T, d)
    #poles2, zeros2 = control.matlab.pzmap(sys_cl, True)
    #sys_cl = control.ss(A-b*(r1), b, c.T, d)
    #poles2, zeros2 = control.matlab.pzmap(sys_cl, True)
    #print p
    sys_cl = con.ss(A-b*(k0_0+1.0/p*k0_1).T, b, c, d)
    pole, zeros = con.matlab.pzmap(sys_cl, True)
    #print pole[0]
    poles.append(pole)
    #print poles2
plt.show

In [ ]:
# Get Polynomcoefficiants
# k1_i -> k_{*,i}

k1_0, k1_1 = k_explizit2(roots_m1_p1, roots_m1_pmin, pmin, A, b)

plt.axis([-3, .1, -4.1, 4.1])
poles = []
for p in np.arange(1, pmin, -0.01):
    #sys_cl = control.ss(A-b*(r0), b, c.T, d)
    #poles2, zeros2 = control.matlab.pzmap(sys_cl, True)
    #sys_cl = control.ss(A-b*(r1), b, c.T, d)
    #poles2, zeros2 = control.matlab.pzmap(sys_cl, True)
    #print p
    sys_cl = con.ss(A-b*(k1_0+1.0/p*k1_1).T, b, c, d)
    pole, zeros = con.matlab.pzmap(sys_cl, True)
    #print pole[0]
    poles.append(pole)
    #print poles2
plt.show

In [ ]:
print "Entering step 2"

In [ ]:
# Transformations A.4

from scipy.special import comb as nchoosek # n Choose k (n ueber k)

# n muss be dim(Q) = dim(R1) = dim(A)
# manual approved with equations
def get_S_list(u_max, Q, z, a, n):
    S_list = [cvxpy.bmat([[u_max**2 , z.T],
                         [z        , Q]])] # S0 is different !
    
    for i in range(n+1)[1:]:
        #print i
        #print -a[n-i]
        q = Q[:, n-i] # Slicing, (n-i)th column of Q!
        #print q
        S_list.append(-a[n-i] * cvxpy.bmat([[0, q.T],
                                            [q, np.zeros((n,n))]]))
    return S_list

# As seen in A.4 jasniwiecz
# Intervalltransformation (Gleichung (4.36))
# p => [p_l, p_u] (p => [p_min, 1]) wird zu p^ => [-1,1] return neue Matrizen(!) mit diesen p^
#
# S_in ist Liste aus Faktormatrizen für Polynom in p
# S_out ist Liste aus Faktormatrizen für Polynom in p^
# a = (p_u - p_l)
# b = (p_u + p_l)
# manual approved with equations
def trans_S_list(S_in, pl=0.1, pu=1):
    a = pu - pl
    b = pu + pl
    n = len(S_in) # Anzahl der Matrizen in S_in

    S_out = [np.zeros(S_in[0].size)] * n
    for j in range(0, n): # Bearbeite jede Matrix
        for i in range(j, n): # Jeweils mit Summe der anderen
            S_out[j] = S_out[j] + 2**(-i) * b**(i-j) * nchoosek(i, j) * S_in[i]
        S_out[j] = a**j*S_out[j]
    return S_out


def calc_S_Sum(S_list):
    l = len(S_list) # number of matrizen in S_list
    m = l-1 # Index of Q_m
    n = S_list[0].size[0] # shape of each matrix, first element
    
    if m is 0:
        S_sum = cvxpy.bmat([[2*S_list[0],   np.zeros(n)], 
                            [np.zeros(n), np.zeros(n)]])
    elif m is 1:
        S_sum = cvxpy.bmat([[2*S_list[0], S_list[1]],
                            [S_list[1],   np.zeros(n)]])
    else: # e.g. m is 2 or more
        S_sum = cvxpy.bmat([[2*S_list[0], S_list[1]],
                            [S_list[1], 2*S_list[2]]])

    for i1 in range(3, l, 2):
        S_new_col = cvxpy.vstack(np.zeros((((i1+1)/2-1)*n, n)), S_list[i1])

        if i1 is m:
            S_new_row = cvxpy.hstack(np.zeros((n, ((i1+1)/2-1)*n)), S_list[i1], np.zeros((n,n)))
        else:
            S_new_row = cvxpy.hstack(np.zeros((n, ((i1+1)/2-1)*n)), S_list[i1], 2*S_list[i1+1])

        S_sum = cvxpy.bmat([[S_sum, S_new_col],
                            [S_new_row]])
    # Beware, there is NO minus compared with Dilyanas version
    S_sum = 0.5*S_sum
    
    return S_sum

# CJ = "Selection matrizes" of (31), l=dimension of P and G
def calc_lmi_cond(S_sum, n):
    k = S_sum.size[1] / n

    J = np.hstack([np.zeros((n*(k-1), n)), np.eye(n*(k-1))])
    C = np.hstack([np.eye(n*(k-1)), np.zeros((n*(k-1), n))])
    
    CJ = np.vstack([C, J])
    l = n*(k-1)
    return CJ, l


In [ ]:
###############################
# Optimisation problem (4.40) #
###############################

# Define Variables
R0 = cvxpy.Variable(n,n)
R1 = cvxpy.Variable(n,n)

G0 = cvxpy.Variable(n,n) # G -> skew
G1 = cvxpy.Variable(n, n) # G_star -> skew

D0 = cvxpy.Variable(n, n) # D -> sym
D1 = cvxpy.Variable(n, n) # D_star - > sym

# Define Parameters
alpha = 1-pmin
beta = 1+pmin

A0 = A-b*k0_0.T                                                  # A^{hat}_0
A1 = A-b*k0_1.T                                                  # A^{hat}_1

S0 = A1.T*R1 + R1*A1                                             # Sigma_0
S1 = A0.T*R1 + R1*A0 + A1.T*R0 + R0*A1                           # Sigma_1
S2 = A0.T*R0 + R0*A0                                             # Sigma_2

A0_star = A-b*k1_0.T                                             # A^{hat}_{*, 0}
A1_star = A-b*k1_1.T                                             # A^{hat}_{*, 1}

S0_star = A1_star.T*R1 + R1*A1_star                              # Sigma_{*,0}
S1_star = A0_star.T*R1 + R1*A0_star + A1_star.T*R0 + R0*A1_star  # Sigma_{*,1}
S2_star = A0_star.T*R0 + R0*A0_star                              # Sigma_{*,2}


# Define constraints

# Constraints on helper variables
constraint_D0 = D0  == D0.T # sym
constraint_D1 = D1  == D1.T # sym

constraint_G0 = G0 + G0.T == 0 # skew
constraint_G1 = G1 + G1.T == 0 # skew

# Constraints (4.39)
constraint_439a = R0 >> 0
constraint_439b = R0 + R1 >> 0

constraint_439c = cvxpy.bmat([[-D0 , G0],
                              [G0.T, D0]]) - \
                  cvxpy.bmat([[S0 + 0.5*beta*S1 + 0.25*beta**2*S2, 0.25*alpha*(S1 + beta*S2)],
                              [0.25*alpha*(S1 + beta*S2),          0.25*alpha**2*S2]]) >> 0

constraints_439d = [1.0 - X0[i].T*(R0 + R1)*X0[i] > 0 for i in range(0, len(X0))]

constraint_439e = cvxpy.bmat([[1,    k0_0.T],
                              [k0_0, R0]]) + \
                     1/pmin * cvxpy.bmat([[0,    k0_1.T],
                                           [k0_1, R1]]) >> 0

constraint_439f = cvxpy.bmat([[1,    k0_0.T],
                              [k0_0, R0]]) + \
                     cvxpy.bmat([[0,    k0_1.T],
                                 [k0_1, R1]]) >> 0

constraint_439g = cvxpy.bmat([[-D1 , G1],
                              [G1.T, D1]]) - \
                 cvxpy.bmat([[S0_star + 0.5*beta*S1_star + 0.25*beta**2*S2_star, 0.25*alpha*(S1_star + beta*S2_star)],
                             [0.25*alpha*(S1_star + beta*S2_star),               0.25*alpha**2*S2_star]]) >> 0


constraints_439 = [constraint_D0, # sym
                   constraint_D1, # sym
                   constraint_G0, # skew
                   constraint_G1, # skew
                   constraint_439a,
                   constraint_439b,
                   constraint_439c] 

constraints_439.extend(constraints_439d) ##!! Beware of the "extend" if input is array

constraints_439.append(constraint_439e)
constraints_439.append(constraint_439f)
constraints_439.append(constraint_439g)

# Form objective.
obj_440 = cvxpy.Minimize(cvxpy.trace(R0+1/pmin*R1))

# Form and solve problem.
prob_440 = cvxpy.Problem(obj_440, constraints_439)

#prob_440.solve(solver=cvxpy.SCS, verbose=True, max_iters=800000)  # Returns the optimal value.
prob_440.solve(solver=cvxpy.MOSEK, verbose=True)  # Returns the optimal value.

print "status:", prob_440.status
print "optimal value", prob_440.value
try:
    print "R0", R0.value
    LA.cholesky(R0.value)
    print "R1", R1.value
    LA.cholesky(R1.value + R0.value)
    print "\n\n-----------------------------------\nSolution ok!"

except:
    print "\n\n-----------------------------------\nSolution not ok!"
    pass